# Serial Optimization
How do you optimise serial code? We cover some of the techniques that you might use for serial optimization in Python: 
1. Vectorization
2. Memoization
3. Compilation

## Vectorization
Because Python is an interpreted language - i.e. each line is dispatched to an interpreter program to interpret and execute - there can be a lot of overhead compared to compiled programs. Python needs to check variable types and use the correct functions for the inputs, because these are not necessarily specified. As a result, when you're using Python for large datasets or for long loops, it can pay to implement vectorization.

Vectorization essentially increases memory locality, and allows the CPU to use SIMD instructions.  Single Instruction, Multiple Data (SIMD) instructions - these allow the same instruction to be performed on multiple cells of data simultaneously. There is therefore much less overhead per operation, providing large speedups. Improving memory locality also reduces the overhead of dereferencing lots of values that are in lots of different places in memory - contiguous memory blocks that contain all the data you need are loaded on to the CPU instead.

Consider the following examples. The first uses base Python to generate a list of integers using a for loop. The second does the same, but uses `numpy` to vectorize the calculation. The `%%timeit` readouts show a large speed increase in the second example.

In [1]:
def double_ints():
  result = []
  for x in range(1_000_000):
    result.append(x * 2)
  return result


In [2]:
%timeit double_ints()

58.1 ms ± 159 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
import numpy as np

In [4]:
def np_double_ints():
  ints = np.arange(1_000_000)
  return ints * 2

In [5]:
assert double_ints() == np_double_ints().tolist(), "Results do not match!"

In [6]:
%timeit np_double_ints()

457 μs ± 8.19 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


We see a large speed increase in the second example, which uses `numpy` to vectorize the calculation.

### Vectorization Exercises
The follow code calculates $\pi$ using Monte Carlo integration. Answer the questions.

The following code, which calculates pi using Monte Carlo integration, is provided. Answer the following questions:
1. Is it correct?
2. How long does it take to run?
3. Where is the most time taken, cumulatively?

Then:
1. Vectorize the code
2. Verify the outputs
3. Measure any speedup.

In [7]:
import random

def monte_carlo_pi(num_samples):
    inside_circle = 0
    for _ in range(num_samples):
        x, y = random.uniform(-1, 1), random.uniform(-1, 1)
        if x**2 + y**2 <= 1:
            inside_circle += 1
    return (inside_circle / num_samples) * 4

**Q1.1: Is `monte_carlo_pi()` correct? At what number of samples are you accurate to three digits?**

<details>
<summary>I need a hint</summary>
Run the function with varying `num_samples` and decide whether, as `num_samples` increases, the output approaches 3.1415
</details>

**Q1.2: How long does the function take to run, on average, when `num_samples = 1_000_000`? How does this scale as you vary the number of samples**

<details>
<summary>I need a hint</summary>

Use the `time` module or use `%timeit`
<details>
<summary>I need a bigger hint</summary>

Write a wrapper function which runs and times the function.
<details>
<summary>Give me the code</summary>

```{Python}
def pi_time(n):
    import time
    start = time.time()
    out = monte_carlo_pi(n)
    end = time.time()
    return (out, end - start)
```
</details>
</details>
</details>



**Q1.3: What functions are called most in `monte_carlo_pi()`?**

<details>
<summary>I need a hint</summary>

Use `cProfile`
<details>
<summary>I need a bigger hint</summary>

Either use `%prun` magic, or call `cProfile()` directly.
<details>
<summary>Give me the code</summary>

```{Python}
%prun monte_carlo_pi(10_000_000)
```

OR

```{Python}
import cProfile
cProfile.run("monte_carlo_pi(1_000_000)")
```
</details>
</details>
</details>

**Q1.4: Where is the most time taken in `monte_carlo_pi()`, cumulatively?**

<details>
<summary>I need a hint</summary>

You want to know which line takes most time.
<details>
<summary>I need a bigger hint</summary>

Use `line_profiler` or `%lprun`.
<details>
<summary>Give me the code</summary>

```{Python}
%load_ext line_profiler
%lprun -f monte_carlo_pi monte_carlo_pi(10_000_000)
```
</details>
</details>
</details>



**Q1.5: Can you speed up the function?**

<details>
<summary>I need a hint</summary>

Previous results should have shown most of the time is taken generating random numbers, and checking whether the random point is inside a circle. Can you vectorise either of these?
<details>
<summary>I need a bigger hint</summary>

`numpy` has functions that vectorise random number generation, and allows you to vectorise exponentiation.
<details>
<summary>Give me the code</summary>

```{Python}
def monte_carlo_pi_numpy(num_samples):
  x, y = np.random.uniform(-1, 1, (2, num_samples)) # vectorised random number generation
  pi = 4 * np.mean(x**2 + y**2 <= 1) # perform check simultaneously on all datav`\` has functions that vectorise random number generation, as wend allows you to vectorise exponentiation.pnumpy
  return pi
```
</details>
</details>
</details>


**Q1.6: Is your edited function correct? Is it faster? Are there any other ways you can think of to speed it up?**

## Memoization
Memoization is a technique to cache results when they're calculated so they can be quickly retrieved when the same input is called. It avoids expensive computation of the same results again and again.

Let's remind ourselves of the function we defined previously to calculate the $n^{th}$ number of the Fibonacci sequence.

In [8]:
def fib(n):
    if n <= 2:
        return 1
    return fib(n - 1) + fib(n - 2)

In [9]:
%timeit fib(35)

894 ms ± 5.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


On our computer, `fib(35)` takes 892 ms on average.

We can implement a cache simply ourselves, using dictionaries.

In [10]:
cache = {}
def fib_cache(n):
    if n in cache:
        return cache[n]
    if n < 2:
        result = n
    else:
        result = fib(n-1) + fib(n-2)
    cache[n] = result
    return result


In [11]:
assert fib(35) == fib_cache(35), "Cached result does not match non-cached result!"
%timeit fib_cache(35)

65.4 ns ± 0.416 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


On our computer, calling `fib_cache(35)` took an average of 61 nanoseconds. Merely implementing a basic cache resulted in approximately a 1-million-fold speedup.

Due to the recursive nature of this function, the speedup achieved will be greater a n increases, due to the greater number of avoided computations as n grows.

We can also use the `functools` module, which comes with Python and provides the function decorator `@lru_cache`. LRU stands for Least Recently Used - when the cache hits its size limit it removes the least recently used data to make room for the new data. 

In [12]:
from functools import lru_cache

@lru_cache(maxsize=None)
def fib_lru(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

In [13]:
%timeit fib_lru(35)

300 ns ± 532 ns per loop (mean ± std. dev. of 7 runs, 1 loop each)


On our computer, the cached version took 257 nanoseconds, *slower* than using our dictionary. Does that hold on your computer? Did you expect it? The functools version avoids code complexity and makes it very simple to cache, however, in this case it does not appear to be as fast as using a dictionary - reiterating the importance of measuring all changes to your code.


### Memoization Exercises

We provide a function below that computes the Binomial Coefficient ("n choose k") recursively.

In [14]:
def binomial(n, k):
    if k == 0 or k == n:
        return 1
    return binomial(n - 1, k - 1) + binomial(n - 1, k)


**Q2.1: How long does this function take with n = 20, k = 10? Explore how the time taken changes as a function of n or k?**

<details>
<summary>I need a hint</summary>
Time the code with a variety of n/k.
<details>
<summary>I need a bigger hint</summary>
Use %timeit, or write a wrapper function.
<details>
<summary>Give me the code</summary>

Edit the code below and explore.
```
%timeit binomial(20, 10)
```
</details>
</details>
</details>


**Q2.2: How many recursive function calls are there with n = 20, k = 10?**

<details>
<summary>I need a hint</summary>

Use `cProfile`.
<details>
<summary>I need a bigger hint</summary>

Either use `cProfile.run()` or `%prun`.
<details>
<summary>Give me the code</summary>

```{Python}
%prun binomial(20, 10)
```
</details>
</details>
</details>


**Q2.3 Implement memoization. How fast is the function with the same inputs you tested before? How many function calls are there?**

<details>
<summary>I need a hint</summary>

Cache the results if not already in the cache.
<details>
<summary>I need a bigger hint</summary>

Either use a dictionary cache, or `@lru_cache` from `functools`, then time it and use `cProfile`.
<details>
<summary>Give me the code</summary>

```{Python}
@lru_cache(maxsize=None)
def binomial(n, k):
    if k == 0 or k == n:
        return 1
    return binomial(n - 1, k - 1) + binomial(n - 1, k)

%timeit binomial(20, 10)

%prun binomial(20, 10)
```
</details>
</details>
</details>

## Compilation of functions
Python, as an interpreted language, has many overheads as we've discussed. To gain code flexibility and speed of implementation, the language had to make many dynamic checks as code is interpreted. The `CPython` interpreter is well implemented, and these checks are individually fairly fast, but over millions of computations the overhead adds up. In addition, there are implementation details that can simply make speedy code impossible - such as Python lists being arrays of pointers meaning every loop over them involves fetching data twice (get the pointer, then get the data where the pointer points to).

There are, however, ways of stepping around the interpreter, and compiling our code ourselves for use in Python. The longest running and most mature method of this is `Cython`, which involves learning a small amount of `C` and setting up a toolchain. We will not be covering that, but if you're interested I encourage you to check it out at https://cython.org/.

We will cover the `Numba` package. `Cython` is an Ahead-Of-Time (AOT) compiler - you compile your function(s) before using, creating a static library that can instantly be used. In comparison, `Numba` is a Just-In-Time (JIT) compiler - when your function is first called it will compile the function then, and *future* uses of that function will benefit from the compiled version. The first use, then, will be slow due to the compilation, with all subsequent calls being (hopefully) much faster.

If you're using `Numba`, you simply add a decorator and away we go. There are two decorators to consider, because `Numba` has a `nopython` mode. In `nopython` mode the function is compiled to machine code without using the Python interpreter *at all* - however, if this is not possible it fails. Avoiding all the interpreter overhead is often worth it, though. 

- `@jit()` - tells `Numba` that you want this function to be compiled. It will try to use `nopython`, but if it fails it will fall back to using the python interpreter. Use `@jit(nopython=True)` to enforce `nopython` mode.
- `@njit()` - tell `Numba` to compile the function in `nopython` mode.

So, lets start using `Numba`. Below we code a Sieve of Eratosthenes using `numpy` to find all primes up to a number `n`. For a primer on the Sieve of Eratosthenes, read the [wikipedia page](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes)

In [15]:
def sieve_primes(n):
    is_prime = np.ones(n + 1, dtype=np.uint8) # Create a boolean array
    is_prime[:2] = 0  # 0 and 1 are not prime numbers
    for i in range(2, int(n**0.5) + 1):
        if is_prime[i] == 1:
            is_prime[i*i:n+1:i] = 0 # Mark multiples of i as non-prime
    return np.where(is_prime)[0] # Get indices of True values, which are the prime numbers

assert len(sieve_primes(10000)) == 1229, "sieve_primes did not return expected number of primes!"

%timeit sieve_primes(100000)



289 μs ± 2.83 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


On our computer, the sieve of eratosthenes took 161 microseconds on average, when n = 100,000. 

Adding the `@jit` decorator:

In [16]:
from numba import jit

@jit(nopython = True)
def sieve_primes_numba(n):
    is_prime = np.ones(n + 1, dtype=np.uint8) # Create a boolean array
    is_prime[:2] = 0  # 0 and 1 are not prime numbers
    for i in range(2, int(n**0.5) + 1):
        if is_prime[i] == 1:
            is_prime[i*i:n+1:i] = 0 # Mark multiples of i as non-prime
    return np.where(is_prime)[0] # Get indices of True values, which are the prime numbers


In [17]:
assert len(sieve_primes_numba(10000)) == 1229, "sieve_primes did not return expected number of primes!"

%timeit sieve_primes_numba(100000) 

205 μs ± 10.3 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


On our computer, simply adding the decorator leads to an average speed of 196 microseconds, approximately a 1/3rd speedup for essentially no effort. Compilation often results in much faster speedups, however, this implementation already used vectorised operations, and the Sieve of Eratosthenes is already fairly optimal.


### Compilation exercises

We revisit here the Julia Set code from the profiling exercises. If you've forgotten them, take a moment to refamiliarise yourself. 


**Q3.1 Remind yourself, how fast was the `calc_z_serial_python()` function before?**


**Q3.2 Use Numba to JIT compile the function. Try with and without nopython mode. How fast is it now?**

<details>
<summary>I need a hint</summary>

Edit the code in the `julia_set.py` file. Use the decorators mentioned and time the use of the function.
<details>
<summary>I need a bigger hint</summary>

Use `@jit()` or `@njit()` on the `calc_z_serial_python()` function, and call the code from the terminal. 
<details>
<summary>Give me the code</summary>

```{Python}
@jit
def calc_z_serial_python(maxiter, zs, c):
  #' Calculate the Julia set for a given set of complex numbers and a constant c.
  output = [0] * len(zs)
  for i in range(len(zs)):
    z = zs[i]
    n = 0
    while abs(z) <= 2 and n < maxiter:
      z = z * z + c
      n += 1
    output[i] = n
  return output
```

In the CLI:

```{python}
python julia_set.py
```
</details>
</details>
</details>


**Q3.3 How long does the compiled function take on the first run? How long on the second?**

<details>
<summary>I need a hint</summary>

Edit the code in the `julia_set.py` file to call the function once or twice. Time them.
<details>
<summary>I need a bigger hint</summary>

You can add time points before and after running the function in `run_julia_set()` Then call `run_julia_set()` twice in `__main__`. 
<details>
<summary>Give me the code</summary>

```{Python}
  start = time.time()
  output = calc_z_serial_python(max_iterations, zs, CONSTVAL) # second run, fully compiled
  end = time.time()


  ...

if __name__ == "__main__":
  run_julia_set(desired_width = 1000, max_iterations = 300)
  run_julia_set(desired_width = 1000, max_iterations = 300)
```

In the CLI:

```{python}
python julia_set.py
```
</details>
</details>
</details>


**Q3.4 Does `run_julia_set()` still spend the majority of its time in the `calc_z_serial_python()` function?**

<details>
<summary>I need a hint</summary>

Use line profiling.
<details>
<summary>I need a bigger hint</summary>

Add the `@profile` decorator to the `run_julia_set()` function and run from the CLI.
<details>
<summary>Give me the code</summary>

```{Python}
from line_profiler import profile
...
...
@profile
def run_julia_set(desired_width, max_iterations):
  ...
```

In the CLI:

```{python}
LINE_PROFILE=1 python julia_set.py
```
</details>
</details>
</details>

**Q3.5 Would you use compilation even though we only (in this example) call the compiled function once?**

<details>
<summary>My thoughts</summary>

On my computer, compiling the `calc_z_serial_python()` function means the function takes 1.02 seconds (including compilation) rather than 3.23 seconds, a 3-fold speedup even with the added compilation time. Post-compilation, it takes 0.76 seconds per call. However, compiling `calc_complex_numbers()` adds more overhead - with no compilation the function takes 0.09 seconds, with compilation overhead it takes 0.23 seconds, and post-compilation it takes 0.04 seconds. If I knew I would only call this once, I'd compile the first function but not the second.

Note that if you *run the script* multiple times, but the script only runs the function once, you incur the compilation cost multiple times as well. There is no substitute for measuring.
</details>